<a href="https://colab.research.google.com/github/phanmanhtung/Natural-Language-Processing-Course/blob/master/Vietnamese_Word_Segmentation_TungPhan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Programming Assignment 4: Vietnamese Word Segmentation

In this assignment, you will need to build a model for Vietnamese Word Segmentation. 

**Rules**:

- You are freely to choose the approach, however, you are required to implement the model by yourself in the Google Colab. 
- **It is not allowed to use available Vietnamese Word Segmentation toolkits to do the assignment.**

**Due date: April 10, 2020 (Friday)**

## Vietnamese Word Segmentation

In Vietnamese, one word may contain multiple syllables (e.g., 'học sinh'). Since there is not delimiter between words, in some applications, we may need to do word segmentation in the preprocessing step.

The input of a Vietnamese Word Segmentation model is a sequence of syllables.
(Note that, you may need to separate punctuations from words in the real scenerio). The output is a word-segmented text.

Example:

- Input: Nam hồn nhiên : " Tụi tôi xài tiền ngân hàng không à " .
- Output: Nam hồn_nhiên : " Tụi tôi xài tiền ngân_hàng không à " .

Similar as named-entity recognition, we can formalize the word segmentation task as a sequence labeling task with BI encoding scheme. We will predict if the tag of a syllable is B-W (begin of a word) or I-W (inside of a word). Tags for the sequence in the above example will be.

```
Nam	B-W
hồn	B-W
nhiên	I-W
:	B-W
"	B-W
Tụi	B-W
tôi	B-W
xài	B-W
tiền	B-W
ngân	B-W
hàng	I-W
không	B-W
à	B-W
"	B-W
.	B-W
```

We can choose Hidden Markov Models, Maximum Entropy Markov Models, or Conditional Random Fields to solve the problem. Please refer to previous lectures and assignments for details about those models.

Student name: Phan Manh Tung
</br>
Student ID: USTHBi8-160


## Dataset

You will use the training data in the file [train.txt](https://www.dl.dropboxusercontent.com/s/reor8jnqedk7svt/train.txt) to train your Vietnamese Word Segmentation Model and evaluate the model on the test data in the file [test.txt](https://www.dl.dropboxusercontent.com/s/zp635cd1zhofm62/test.txt) derived from VLSP 2013 Word Segmentation dataset.

You can download the file using wget command.

In [0]:
!rm -f train.txt
!wget https://www.dl.dropboxusercontent.com/s/reor8jnqedk7svt/train.txt

--2020-04-10 15:16:43--  https://www.dl.dropboxusercontent.com/s/reor8jnqedk7svt/train.txt
Resolving www.dl.dropboxusercontent.com (www.dl.dropboxusercontent.com)... 162.125.82.6, 2620:100:6032:6::a27d:5206
Connecting to www.dl.dropboxusercontent.com (www.dl.dropboxusercontent.com)|162.125.82.6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5485066 (5.2M) [text/plain]
Saving to: ‘train.txt’

train.txt           100%[===================>]   5.23M  3.62MB/s    in 1.4s    

2020-04-10 15:16:45 (3.62 MB/s) - ‘train.txt’ saved [5485066/5485066]



In [0]:
!head -3 train.txt

Nam	B-W
hồn	B-W
nhiên	I-W


In [0]:
from collections import defaultdict

def train(train_file: str, model_file: str):
    emit = defaultdict(int)   # dictionary to store emission count C(t_i, w_i)
    transition = defaultdict(int)   # transition count C(t_{i-1}, t_i)
    context = defaultdict(int)  # count the context
    with open("train.txt", 'r') as f:
      previous = '<s>'    # Make the document start
      context[previous] += 1
      for line in f:
        if line != "\n":
          line = line.strip()
          word, tag = line.split()
          transition[previous+" "+tag] += 1 # previous and the next tag (eg: B-W -> I-W)
          context[tag] += 1
          emit[tag+' '+word] += 1 # certain words within 1 tag
          previous = tag

    # Now we will save the parameters of the model to a file
    with open(model_file, 'w') as fo:

        # Save transition probabilities
        for key, value in transition.items():
            previous, word = key.split(' ')
            fo.write('T %s %f\n' % (key, value/context[previous]))

        # Write emission probabilities into the model file

        # Save emission probabilities
        for key, value in emit.items():
            tag, word = key.split(' ')
            fo.write('E %s %f\n' % (key, value/context[tag]))

    print('Finished training first-order HMM!')

In [0]:
train('train.txt', 'HMM_model.txt')

Finished training first-order HMM!


In [0]:
!head HMM_model.txt

T <s> B-W 1.000000
T B-W B-W 0.749816
T B-W I-W 0.250182
T I-W B-W 0.944108
T I-W I-W 0.055892
E B-W Nam 0.000467
E B-W hồn 0.000033
E I-W nhiên 0.003117
E B-W : 0.005324
E B-W " 0.015124


In [0]:
!tail HMM_model.txt

E B-W JOMSRE 0.000002
E B-W 6,1 0.000002
E B-W Hủ 0.000002
E I-W kị 0.000008
E B-W 08:36:27 0.000002
E I-W úp 0.000008
E B-W 470 0.000002
E B-W Sàn 0.000002
E I-W Hàng 0.000008
E B-W Dỉnh 0.000002


In [0]:
# Load model

def load_model(model_file: str):
    """Load saved HMM model
    """
    transition = defaultdict(lambda: 0)
    emission = defaultdict(lambda: 0)
    possible_tags = {}
    
    # TODO: Write your code to load the model file
    # YOUR CODE HERE
    with open(model_file, 'r') as f:
      for line in f:
        line = line.strip()
        T_E, context, word, prob = line.split(" ")
        possible_tags[context] = 1
        if (T_E == 'T'):
          transition[context + ' ' + word] = prob
        elif (T_E == 'E'):
          emission[context + ' ' + word] = prob
    
    # END OF YOUR CODE HERE
    return transition, emission, possible_tags

In [0]:
transition, emission, possible_tags = load_model('HMM_model.txt')
print( list(possible_tags.keys()) )
print(transition)
print(emission)

['<s>', 'B-W', 'I-W']
defaultdict(<function load_model.<locals>.<lambda> at 0x7f7bfe420ae8>, {'<s> B-W': '1.000000', 'B-W B-W': '0.749816', 'B-W I-W': '0.250182', 'I-W B-W': '0.944108', 'I-W I-W': '0.055892'})
defaultdict(<function load_model.<locals>.<lambda> at 0x7f7bfe3cbae8>, {'B-W Nam': '0.000467', 'B-W hồn': '0.000033', 'I-W nhiên': '0.003117', 'B-W :': '0.005324', 'B-W "': '0.015124', 'B-W Tụi': '0.000022', 'B-W tôi': '0.002298', 'B-W xài': '0.000033', 'B-W tiền': '0.001633', 'B-W ngân': '0.000389', 'I-W hàng': '0.003150', 'B-W không': '0.007544', 'B-W à': '0.000037', 'B-W .': '0.033665', 'B-W Chỉ': '0.000261', 'B-W trong': '0.006844', 'B-W năm': '0.003983', 'B-W 2001': '0.000085', 'B-W Tomb': '0.000004', 'I-W Raider': '0.000016', 'B-W đã': '0.008170', 'B-W mang': '0.000587', 'B-W về': '0.004757', 'B-W cho': '0.007872', 'B-W Hollywood': '0.000007', 'B-W 300': '0.000139', 'B-W triệu': '0.001133', 'B-W USD': '0.000496', 'B-W (': '0.006148', 'B-W dân': '0.001626', 'B-W ghiền': '0.0

In [0]:
def viterbi(line, transition, emission, possible_tags):
    words = line.split()
    l = len(words)
    best_score = {}
    best_edge = {}
    best_score[('0 <s>')] = 0  # Start with <s>
    best_edge[('0 <s>')] = None
    # Forward Step
    for i in range(l):
        for prev in possible_tags.keys():
            for _next in possible_tags.keys():
                if str(i) + ' ' + prev in best_score and prev + ' ' + _next in transition:
                    if emission[_next + ' ' + words[i]] == 0:
                        # To avoid zero probabilities, we use very small value
                        emission[_next + " " + words[i]] = 10 ** (-10)
                    # TODO: Write code to calculate the score of the path that
                    # connect the best path in the step i with the edge prev -> _next
                    # score = ...

                    score = best_score[str(i) + ' ' + prev] + -math.log(float(transition[prev + ' ' + _next])) + -math.log(float(emission[_next + ' ' + words[i]]))

                    # END OF YOUR CODE HERE
                    if str(i + 1) + " " + _next not in best_score or best_score[str(i + 1) + " " + _next] > score:
                          best_score[str(i + 1) + " " + _next] = score
                          best_edge[str(i + 1) + " " + _next] = str(i) + " " + prev

    tags = []
    next_edge = best_edge[str(l) + " " + "B-W"]
    # TODO: Complete the backward step in Viterbi algorithm
    # Finish the while loop in the pseudo code
    while (next_edge != '0 <s>'):
      position,tag = next_edge.split(' ')
      tags.append(tag)
      next_edge = best_edge[next_edge]

    tags.reverse()
    return ' '.join(tags)+ " B-W"

In [0]:
# Try out with some examples:
viterbi('Hôm nay là ngày đầu tuần , bé hứa cố gắng chăm ngoan .', transition, emission, possible_tags)

'B-W I-W B-W B-W B-W B-W B-W B-W B-W B-W I-W B-W I-W B-W'

In [0]:
viterbi('Thứ ba , thứ tư , thứ năm , ngày nào cũng luôn ngu ngốc nhưng bảnh bao .', transition, emission, possible_tags)

'B-W B-W B-W B-W I-W B-W B-W B-W B-W B-W B-W B-W B-W B-W B-W B-W B-W B-W B-W'

In [0]:
# Create output file for evaluation

result1 = viterbi('Tôi là sinh viên .', transition, emission, possible_tags)
print(result1)

B-W B-W B-W I-W B-W


In [0]:
result2 = viterbi('Tôi là giảng viên đại học .', transition, emission, possible_tags)
print(result2)

B-W B-W B-W I-W B-W I-W B-W


In [0]:
line1 = 'Tôi là sinh viên .'
line2 = 'Tôi là giảng viên đại học .'

for i in range(len(line1.split())):
  print(line1.split()[i] + "    " + result1.split()[i])

print("\n")

for i in range(len(line2.split())):
  print(line2.split()[i] + "    " + result2.split()[i])

Tôi    B-W
là    B-W
sinh    B-W
viên    I-W
.    B-W


Tôi    B-W
là    B-W
giảng    B-W
viên    I-W
đại    B-W
học    I-W
.    B-W


In [0]:
my_output = '''Tôi    B-W
là    B-W
sinh    B-W
viên    I-W
.    B-W


Tôi    B-W
là    B-W
giảng    B-W
viên    I-W
đại    B-W
học    I-W
.    B-W
'''


f=open('file.txt', 'w')
f.write(my_output)

133

In [0]:
!cat file.txt

Tôi    B-W
là    B-W
sinh    B-W
viên    I-W
.    B-W


Tôi    B-W
là    B-W
giảng    B-W
viên    I-W
đại    B-W
học    I-W
.    B-W


The training data contains 20000 sentences (sentences are separated by a blank line), and the test data contains 2000 sentences.

## Evaluation Measures

- P(recision): (number of words correctly segmented)/(number of words in the system output)
- R(ecall): (number of words correctly segmented)/(number of words in the reference corpus)
- F1 measure = 2*P*R/(P+R)


### Evaluation code

You will use the following code for your evaluation. The input of the function is two files:

- test_file is the file with gold word segmentation.
- output_file is the the output file which is generated by your system. output_file and test_file have the same format.

The function will return precision, recall, and f1 score.

We will use the package [seqeval](https://github.com/chakki-works/seqeval) for evaluation.

In [0]:
!pip install seqeval[cpu]

  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=b245abcb4a16044cd1a0a7747426c84c0236b63a1e10cb1d1ba369df8d5a12a4
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
Successfully built seqeval


Let's run the evaluate the model on two sample files:

- [answer.txt](https://www.dl.dropboxusercontent.com/s/uhzyv8ofrsoxepo/answer.txt)
- [output.txt](https://www.dl.dropboxusercontent.com/s/7g6w12i59srqqw4/output.txt)

The content of answer.txt and output.txt are as follows.

answer.txt

```
Tôi	B-W
là	B-W
sinh	B-W
viên	I-W
.	B-W

Tôi	B-W
là	B-W
giảng	B-W
viên	I-W
đại	B-W
học	I-W
.	B-W
```

output.txt
```
Tôi	B-W
là	B-W
sinh	B-W
viên	B-W
.	B-W

Tôi	B-W
là	B-W
giảng	B-W
viên	B-W
đại	B-W
học	I-W
.	B-W
```

In [0]:
!rm -f answer.txt
!rm -f output.txt
!wget https://www.dl.dropboxusercontent.com/s/uhzyv8ofrsoxepo/answer.txt
!wget https://www.dl.dropboxusercontent.com/s/7g6w12i59srqqw4/output.txt

--2020-04-10 15:46:24--  https://www.dl.dropboxusercontent.com/s/uhzyv8ofrsoxepo/answer.txt
Resolving www.dl.dropboxusercontent.com (www.dl.dropboxusercontent.com)... 162.125.82.6, 2620:100:6032:6::a27d:5206
Connecting to www.dl.dropboxusercontent.com (www.dl.dropboxusercontent.com)|162.125.82.6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 109 [text/plain]
Saving to: ‘answer.txt’

answer.txt          100%[===================>]     109  --.-KB/s    in 0s      

2020-04-10 15:46:25 (15.3 MB/s) - ‘answer.txt’ saved [109/109]

--2020-04-10 15:46:27--  https://www.dl.dropboxusercontent.com/s/7g6w12i59srqqw4/output.txt
Resolving www.dl.dropboxusercontent.com (www.dl.dropboxusercontent.com)... 162.125.82.6, 2620:100:6032:6::a27d:5206
Connecting to www.dl.dropboxusercontent.com (www.dl.dropboxusercontent.com)|162.125.82.6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 108 [text/plain]
Saving to: ‘output.txt’

output.txt          100%[====

In [0]:
!cat output.txt

Tôi	B-W
là	B-W
sinh	B-W
viên	B-W
.	B-W

Tôi	B-W
là	B-W
giảng	B-W
viên	B-W
đại	B-W
học	I-W
.	B-W

In [0]:
!pip install seqeval

  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=1ff2892e051a3c93e6482d251788b464991bb7064748ca77e0ef01186da1c34e
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
Successfully built seqeval


In [0]:
from seqeval.metrics import precision_score, recall_score, f1_score

def get_tags(filepath):
    res = []
    with open(filepath, 'r') as f:
        cur_sen = []
        for line in f:
            line = line.strip()
            if line == '':
                if len(cur_sen) != 0:
                    res.append(cur_sen)
                    cur_sen = []
            else:
                word, tag = line.split()
                cur_sen.append(tag)
    if len(cur_sen) != 0:
        res.append(cur_sen)
    return res

def evaluate(test_file, output_file):
    y_true = get_tags(test_file)
    y_pred = get_tags(output_file)

    p = precision_score(y_true, y_pred)
    r = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    return p, r, f1

In [0]:
evaluate('./answer.txt', './file.txt')

(1.0, 1.0, 1.0)

## What to submit

- In the Google Colab, you have to report evaluation metrics of your model on the provided test data.
- Your Google Colab should self-contained. I should reproduce the evaluation metrics without your help.
- Send the link to your Google Colab that contains your implementation of Vietnamese Word Segmentation model to my email address (minhpham0902@gmail.com). Please make sure that you share the access permission to me.
- Write your name and student id in the Google Colab.


## References

- Huyen, N. T. M., Roussanaly, A., & Vinh, H. T. (2008, March). A hybrid approach to word segmentation of Vietnamese texts. In International Conference on Language and Automata Theory and Applications (pp. 240-249). Springer, Berlin, Heidelberg.
- Nguyen, T. P., & Le, A. C. (2016, November). [A hybrid approach to Vietnamese word segmentation](https://www.researchgate.net/profile/Tuan_Phong_Nguyen/publication/311980397_A_hybrid_approach_to_Vietnamese_word_segmentation/links/5a9507e3a6fdccecff0771ff/A-hybrid-approach-to-Vietnamese-word-segmentation.pdf). In 2016 IEEE RIVF International Conference on Computing & Communication Technologies, Research, Innovation, and Vision for the Future (RIVF) (pp. 114-119). IEEE.
- Nguyen, D. Q., Nguyen, D. Q., Vu, T., Dras, M., & Johnson, M. (2017). [A fast and accurate vietnamese word segmenter](https://arxiv.org/abs/1709.06307). arXiv preprint arXiv:1709.06307.
- [seqeval](https://github.com/chakki-works/seqevalhttps://github.com/chakki-works/seqeval) for sequence labeling evaluation.

